In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [3]:
## Load the Dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Preprocess the data
## Drop irrelevant features

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)
data.tail()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1
9999,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


In [5]:
## Encode Categorical Variables

Le_gender = LabelEncoder()
data['Gender'] = Le_gender.fit_transform(data['Gender'])
data.tail()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1
9999,792,France,0,28,4,130142.79,1,1,0,38190.78,0


In [6]:
## Applying the One Hot Encoder to Geography column

from sklearn.preprocessing import OneHotEncoder

OHE_Geo = OneHotEncoder()
geo_encoder = OHE_Geo.fit_transform(data[['Geography']])

In [7]:
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [9]:
OHE_Geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns = OHE_Geo.get_feature_names_out(['Geography'])) 

In [11]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [12]:
## Combine all the One hot encoded columns with original data

data = pd.concat([data.drop('Geography', axis = 1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
## Save the Encoders and scaler

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(Le_gender, file)

with open('onehot_encoder_Geo.pkl', 'wb') as file:
    pickle.dump(OHE_Geo, file)

In [14]:
## Divide the dataset into independent and dependent features

X = data.drop('Exited', axis=1)
y = data['Exited']

## Split the data in training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scale these features
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
X_train_scaled

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [16]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN Implementation

In [17]:
import tensorflow as ts
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

2024-11-03 11:36:51.217819: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
(X_train.shape[1],)

(12,)

In [19]:
## Build our ANN model

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## HL1 connected with input layer
    Dense(32, activation='relu'), ## HL2
    Dense(1, activation = 'sigmoid') ## Output Layer
])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
import tensorflow

opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

In [22]:
## Compile the model

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [23]:
## Setup the Tensorboard

log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [24]:
## Set up Early Stopping

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [25]:
## Train the model

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,
                    callbacks = [tensorflow_callback, early_stopping_callback])

Epoch 1/100
250/250 [==============================] - 2s 5ms/step - loss: 252.8008 - accuracy: 0.6911 - val_loss: 92.0947 - val_accuracy: 0.3935
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 121.7772 - accuracy: 0.6856 - val_loss: 78.8912 - val_accuracy: 0.4060
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 99.9413 - accuracy: 0.6784 - val_loss: 43.5687 - val_accuracy: 0.7525
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 79.2357 - accuracy: 0.6785 - val_loss: 71.6133 - val_accuracy: 0.8040
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 74.9292 - accuracy: 0.6848 - val_loss: 108.3761 - val_accuracy: 0.4530
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 73.7401 - accuracy: 0.6798 - val_loss: 101.5020 - val_accuracy: 0.7890
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 79.1045 - accuracy: 0.6810 - val_loss: 36.6665 - va

In [26]:
model.save('model.h5')

/Users/user/Desktop/Chaitanya Study/project/DeepLearning_Project/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [27]:
## Load Tensorboard Extension

%load_ext tensorboard

In [31]:
%tensorboard --logdir logs/fit20241103-113710

Reusing TensorBoard on port 6008 (pid 87070), started 0:00:39 ago. (Use '!kill 87070' to kill it.)

In [ ]:
## Load the pickle file

